## MultiOn Demo

This notebook walks through an example of using LlamaIndex with MultiOn to browse the web on the users behalf.

First, we import the OpenAIAgent that will control the Multion session:

In [1]:
# Set up OpenAI
import openai
from llama_index.agent import OpenAIAgent
openai.api_key = 'sk-your-key'

We then import the MultiOn tool and initialize our agent with the tool. When the MultiOn tool is initialized, the user will be prompted to authenticate the session in their browser

In [2]:
# Set up MultiOn tool
from llama_hub.tools.multion.base import MultionToolSpec
multion_tool = MultionToolSpec()

Already logged in.


To support the MultiOn browsing session, we will also give our LlamaIndex agent a tool to search and summarize a users gmail inbox. We set up that tool below. For more information on the gmail tool, see the [Gmail notebook here](https://github.com/emptycrown/llama-hub/blob/main/llama_hub/tools/notebooks/gmail.ipynb)

In [3]:
# Import and initialize our tool spec
from llama_hub.tools.gmail.base import GmailToolSpec
from llama_index.tools.ondemand_loader_tool import OnDemandLoaderTool

# Initialize the Gmail tool to search our inbox 
gmail_tool = GmailToolSpec()

# Wrap the tool so we don't overflow the main Agent's context window
gmail_loader_tool = OnDemandLoaderTool.from_tool(
    gmail_tool.to_tool_list()[1],
    name="gmail_search",
    description="""
        This tool allows you to search the users gmail inbox and give directions for how to summarize or process the emails

        You must always provide a query to filter the emails, as well as a query_str to process the retrieved emails.
        All parameters are required
        
        If you need to reply to an email, ask this tool to build the reply directly
        Examples:
            query='from:adam subject:dinner', max_results=5, query_str='Where are adams favourite places to eat'
            query='dentist appointment', max_results=1, query_str='When is the next dentist appointment'
            query='to:jerry', max_results=1, query_str='summarize and then create a response email to jerrys latest email'
            query='is:inbox', max_results=5, query_str='Summarize these emails'
        """
)


In [4]:
# Initialize our Agent with the MultiOn and Gmail loader tool
agent = OpenAIAgent.from_tools(
    [*multion_tool.to_tool_list(), gmail_loader_tool],
    system_prompt="""
    You are an AI agent that assists the user in crafting email responses based on previous conversations.
    
    The gmail_search tool connects directly to an API to search and retrieve emails, and answer questions based on the content.
    The browse tool allows you to control a web browser with natural language to complete arbitrary actions on the web.
    
    Use these two tools together to gain context on past emails and respond to conversations for the user.
    """,
    verbose=True,
)

Our agent is now set up and ready to browse the web!

To start we ask the agent to obtain the most recent blog post from openai

In [5]:
print(agent.chat('what are my recent emails about'))

=== Calling Function ===
Calling function: gmail_search with args: {
  "query": "is:inbox",
  "max_results": 5,
  "query_str": "Summarize these emails"
}
Got output: The emails contain information about different topics. The first email is about reading material for developers and technologists, while the second email is about making the most of summer days in San Francisco.
Here is a summary of your recent emails:

1. Email 1: The email is about reading material for developers and technologists.
2. Email 2: The email is about making the most of summer days in San Francisco.

Let me know if you would like more information about any specific email.


In [6]:
print(agent.chat('browse to the latest email from canva'))

=== Calling Function ===
Calling function: browse with args: {
  "instruction": "Go to Gmail and search for the latest email from Canva"
}
running new session
Server Disconnected. Please press connect in the Multion extension popup
Server Disconnected. Please press connect in the Multion extension popup
Got output: {'url': 'https://mail.google.com/mail/u/0/', 'status': 'CONTINUE', 'action_completed': 'I am navigating to Gmail to search for the latest email from Canva.\n\n', 'content': "Mail\n\nChat\n\nfe}\nrR\n\nSpaces\n\nOr\n\nMeet\n\nMi Gmail\n\n@ Compose\n\nfk Inbox\nyx © Starred\n© _ Snoozed\n—& Sent\n\nv More\n\nQ. Search in mail\n\nO-\n\nCG\n\nWorkable\n\nMedium - Programmi.\n\nMedium Daily Digest\nQuickNode\n\nYelp\n\nLearnWeb3\n\nShopify Research\nMedium Daily Digest\nDeb at Zapier\nMedium Daily Digest\nMedium\n\nMedium\n\nWorkable\n\nMedium Daily Digest\nCanva\n\nYelp\n\nMedium\n\nWorkable\n\nMedium\n\nMedium Daily Digest\nDigitalOcean\nMedium Daily Digest\n\n@rctver @ @ # Goo

In [7]:
print(agent.chat('Summarize the email, then create an approriate reply email to it, and then pass the full reply to the browser'))

=== Calling Function ===
Calling function: gmail_search with args: {
  "query": "from:canva",
  "max_results": 1,
  "query_str": "Summarize and create a reply email to the latest email from Canva"
}
Got output: The latest email from Canva is celebrating their 10th birthday and thanking the recipient for their support in shaping a decade of design. To create a reply email, you can express your congratulations and appreciation for Canva's milestone. You can also mention how Canva has positively impacted your design experience and express your excitement for future collaborations.
=== Calling Function ===
Calling function: browse with args: {
  "instruction": "Go to Gmail and compose a new email. Address it to Canva. Subject: Congratulations on your 10th birthday! Body: Dear Canva, Congratulations on reaching this incredible milestone of 10 years! I wanted to express my appreciation for the amazing design platform you have created. Canva has truly transformed my design experience and made